In [9]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ["OPENAI_MODEL_NAME"] = (
    "gpt-4o"  # we need a larger context window for this project
)

In [10]:
from utils import get_google_api_client

google_api_client = get_google_api_client()

In [11]:
from crewai_tools import FileReadTool, SerperDevTool

transcript_path = "kt_transcripts/660.txt"

file_read_tool = FileReadTool(transcript_path)
search_tool = SerperDevTool()

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer
from crewai_tools import Tool
import nltk

nltk.download("vader_lexicon")


def analyze_audience_reaction() -> str:
    """
    Analyzes the audience's reaction in a Kill Tony episode transcript to gauge joke success.
    Returns a brief report on what made the comedian's set successful or unsuccessful.
    """
    # Use NLTK's SentimentIntensityAnalyzer to get sentiment scores
    sia = SentimentIntensityAnalyzer()

    # Read the transcript from the file
    with open(transcript_path, "r") as file:
        transcript = file.read()

    # Split transcript into lines
    lines = transcript.split("\n")

    # Analyze sentiment for lines containing 'Audience'
    audience_reactions = []
    for line in lines:
        if "Audience" in line:
            scores = sia.polarity_scores(line)
            audience_reactions.append((line, scores))

    # Summarize the analysis
    report = "Audience Reaction Analysis:\n"
    for reaction, scores in audience_reactions:
        if scores["compound"] >= 0.5:
            report += f"Positive reaction: {reaction}\n"
        elif scores["compound"] <= -0.5:
            report += f"Negative reaction: {reaction}\n"
        else:
            report += f"Neutral reaction: {reaction}\n"

    return report


analyze_audience_reaction_tool = Tool(
    name="Analyze Audience Reaction",
    func=analyze_audience_reaction,
    description="Analyzes the audience's reaction in a Kill Tony episode transcript to gauge joke success.",
)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/deonna/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [13]:
from crewai import Agent

hollywood_agent = Agent(
    role="Hollywood Agent",
    goal="Identify and promote rising star comedians",
    backstory="""
    You are a successful Hollywood Agent that helps find fresh talent for their first Netflix special.
    You are always on the lookout for promising new talent.
    """,
    verbose=True,
)

comedy_journalist = Agent(
    role="Comedy Journalist",
    goal="Write engaging, humorous recaps of comedy specials",
    backstory="""
    You are a seasoned comedy journalist known for your witty and insightful coverage.
    """,
    verbose=True,
)

aspiring_comedian = Agent(
    role="Aspiring Stand-up Comedian",
    goal="Provide insights on what makes a great 1-minute comedy set",
    backstory="""
    You are an aspiring comedian who studies comedy sets to improve your own craft.
    """,
    verbose=True,
)

accuracy_checker = Agent(
    role="Accuracy Checker",
    goal="Ensure the accuracy and completeness of Kill Tony episode recaps",
    backstory="""
    You are a meticulous fact-checker with a deep knowledge of Kill Tony's history and inside jokes.
    """,
    verbose=True,
)

In [14]:
from crewai import Task

recap_task = Task(
    description="Write a recap of the latest Kill Tony episode, highlighting the best moments and jokes using the transcript",
    expected_output="A humorous 3-paragraph recap of the episode with references to standout jokes and performances",
    output_file="L5_output/recap_task_results.md",
    agent=comedy_journalist,
    async_execution=True,  # This task will be executed asynchronously
    tools=[file_read_tool, analyze_audience_reaction_tool],
)

accuracy_task = Task(
    description="Review the recap for accuracy and completeness, providing suggestions for improvement",
    expected_output="An accuracy score out of 5 and a list of suggestions to enhance the recap",
    agent=accuracy_checker,
    output_file="L5_output/accuracy_task_results.md",
    context=[recap_task],  # Will wait for the output of this task before executing
    tools=[file_read_tool, analyze_audience_reaction_tool],
)

analysis_task = Task(
    description="Analyze the panelists (especially Tony) and audience's reaction to the comedians' sets in the selected episodes using the transcript",
    expected_output="A brief report on what made each comedian's set successful or unsuccessful",
    output_file="L5_output/analysis_task_results.md",
    agent=aspiring_comedian,
    context=[recap_task],  # Will wait for the output of this task before executing
    tools=[file_read_tool, analyze_audience_reaction_tool],
)

identify_talent_task = Task(
    description="Identify promising comedians from the analyzed episodes who could be potential stars",
    expected_output="A list of comedians with standout performances and potential for success",
    output_file="L5_output/identify_talent_task_results.md",
    agent=hollywood_agent,
    context=[analysis_task],  # Will wait for the output of this task before executing
)

In [15]:
from pydantic import BaseModel


# Define a Pydantic model for venue details
class VenueDetails(BaseModel):
    name: str
    address: str
    suitability: str

In [16]:
from crewai_tools import ScrapeWebsiteTool

scrape_tool = ScrapeWebsiteTool()

venue_scout = Agent(
    role="Venue Scout",
    goal="Find comedy venues in Los Angeles for promising comedians",
    backstory="""
    You are a venue scout tasked with finding the best venues and open mics in {city}.
    You have an excellent understanding of venues that would be great for up-and-coming comedians,
    even if they aren't explicitly comedy clubs.
    """,
    verbose=True,
)

find_venue_task = Task(
    description="Research and recommend venues in Los Angeles for promising comedians to perform at",
    expected_output="A list of venues in {city} with details on their name, address, and suitability for open mic performances",
    output_json=VenueDetails,  # Use the Pydantic model to create structured output
    output_file="L5_output/find_venue_task_results.json",
    agent=venue_scout,
    async_execution=True,  # This task will be executed asynchronously
    tools=[search_tool, scrape_tool],
)

In [17]:
from crewai import Crew, Process

crew = Crew(
    agents=[
        hollywood_agent,
        comedy_journalist,
        aspiring_comedian,
        accuracy_checker,
    ],
    tasks=[
        recap_task,
        accuracy_task,
        analysis_task,
        identify_talent_task,
        find_venue_task,
    ],
    verbose=True,
)

2024-05-26 13:32:22,974 - 8528120832 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [18]:
result = crew.kickoff(inputs={"city": "Los Angeles"})

 [DEBUG]: == Working Agent: Comedy Journalist
 [INFO]: == Starting Task: Write a recap of the latest Kill Tony episode, highlighting the best moments and jokes using the transcript
 [DEBUG]: == [Comedy Journalist] Task output: 


 [DEBUG]: == Working Agent: Accuracy Checker
 [INFO]: == Starting Task: Review the recap for accuracy and completeness, providing suggestions for improvement


> Entering new CrewAgentExecutor chain...
To write an engaging and humorous recap of the latest Kill Tony episode, I'll need to first read the transcript of the episode to understand the jokes and performances. Then, I will analyze the audience's reaction to gauge the success of the jokes. These two steps will provide me with the necessary context to write a witty and insightful recap.

Action: Read a file's content
Action Input: {"filename": "kt_transcripts/660.txt"} 

[Music]
[Music]
[Music]
[Music]
n
[Music]
[Music]
[Music]
hey this is Redman coming to you live
from the comedy Mothership here in
Aust

In [19]:
from IPython.display import Markdown, display

display(Markdown(accuracy_task.output_file))


To find the best venues in Los Angeles for up-and-coming comedians, I need to identify places that host open mics and comedy nights. I'll start by searching the internet for venues known for such events.

Action: Search the internet
Action Input: {"search_query": "best venues for open mic comedy in Los Angeles"} 


Search results: Title: L.A. Stand Up Comedy Open Mics
Link: https://lastandups.com/l-a-stand-up-comedy-open-mics/
Snippet: L.A. Stand Up Comedy Open Mics · COMEDY BLVD 7924 Beverly Blvd Los Angeles, CA 90048. Good Vibe Hosted by Ally Noel Time: 5-7pm $5 for 5 Minutes · CAFÉ MUSE 6547 ...
---
Title: Any good Comedy open mic recommendations? : r/LosAngeles
Link: https://www.reddit.com/r/LosAngeles/comments/192lxtz/any_good_comedy_open_mic_recommendations/
Snippet: Anyone know of any fun venues, bars, or clubs that have spots for beginners?
---
Title: LA open mic list - Ian Ira Rousso
Link: https://www.ianirarousso.com/la-open-mic-lists
Snippet: ... best damned bar on the plan

**Accuracy Score:** 4.5 out of 5

**Suggestions to Enhance the Recap:**

1. **Band Members Accuracy:** The recap incorrectly mentions only Carlos Sosa and Michael Gonzalez, while the full band includes Carlos Sosa on the horns, Raul Vallejo and Fernando Castillo, Michael Gonzalez on the drums, Nick Lewis on the bass, and Matt Mueling on the electric guitar. John Deas is the band leader on the keys.

2. **Guest Identification:** The recap refers to “Elaine” and “Ian Edwards,” but it should be clarified that Elaine is a new comedian discovered by Tony and it’s her first time on the panel, which is a significant detail.

3. **Audience Interaction:** The recap mentions Elaine losing a nail and gifting it to an eager audience member, which is correct, and it was a little Mexican boy who received it, adding a humorous detail.

4. **Panel Dynamics:** The interaction between Ian Edwards and Elaine in terms of jokes about Elaine’s age and the documentary joke "That's My Shit" should be highlighted as it adds to the humor and dynamic of the episode.

5. **Regulars' Segments:** The segments of Cam Patterson, Hans Kim, and William Montgomery are well captured, but adding specific jokes or themes they mentioned would provide more depth. For example, Cam Patterson's jokes about Utah's Civil War allegiance and mountain sightings, Hans Kim’s bits about Asian women in San Francisco, and William Montgomery’s story about a racist dead squirrel in his chimney.

6. **Special Appearances and Final Performances:** The recap should mention the special appearance of Heath Cordis and the return of Hans Kim after a long hiatus, as well as the closing performance by William Montgomery, which are significant highlights of the episode.

Here is the enhanced recap with the suggestions incorporated:

---

What a whirlwind of a Kill Tony episode that was! From the moment Redban kicked things off at the comedy Mothership in Austin, Texas, you could feel the electric anticipation in the air. The band, featuring talents like Carlos Sosa on the horns, Raul Vallejo and Fernando Castillo, Michael Gonzalez on the drums, Nick Lewis on the bass, Matt Mueling on the electric guitar, and led by John Deas on the keys, set the stage for an action-packed night. Tony Hinchcliffe, with his typical flair, promised an unforgettable show, and boy, did it deliver.

Elaine and Ian Edwards joined the panel, bringing in a mix of seasoned comedy wisdom and fresh perspective. Elaine, who has been performing stand-up for 50 years, instantly charmed everyone despite losing a nail on stage, which she graciously gifted to a little Mexican boy in the audience. Ian Edwards, always the sharp wit, made cracks about Elaine's age and joked about his documentary "That's My Shit" that had everyone roaring. The episode's energy peaked when Casey Rocket hit the stage with his unique brand of humor, complete with wild eyes and tales of drug-induced flashbacks at the mall. His physical comedy had the audience in stitches, proving why he's quickly becoming a Kill Tony favorite.

But let's not forget the regulars who never fail to bring the house down. Cam Patterson's observations about Utah's Civil War allegiance and mountain sightings were both insightful and hilarious. Meanwhile, Hans Kim made a triumphant return after a hiatus, tackling topics from Asian women in San Francisco to the pitfalls of sending nudes. And who could overlook William Montgomery, who closed the show with his signature blend of absurdity and sharp commentary, even managing to weave in a tale about a racist dead squirrel in his chimney. Every joke hit, every performance sizzled, and the audience was left wanting more. This episode truly showcased why Kill Tony remains the number one live podcast in the world.

---

This enhanced recap captures more details and humor from the episode, providing a fuller picture of the night's events.

In [20]:
display(Markdown(recap_task.output_file))

What a whirlwind of a Kill Tony episode that was! From the moment Redban kicked things off at the comedy Mothership in Austin, Texas, you could feel the electric anticipation in the air. The band, featuring talents like Carlos Sosa on the horns and Michael Gonzalez on the drums, set the stage for an action-packed night. Tony Hinchcliffe, with his typical flair, promised an unforgettable show, and boy, did it deliver.

Elaine and Ian Edwards joined the panel, bringing in a mix of seasoned comedy wisdom and fresh perspective. Elaine, who has been performing stand-up for 50 years, instantly charmed everyone despite losing a nail on stage (which she graciously gifted to an eager audience member). Ian Edwards, always the sharp wit, made cracks about Elaine's age that had everyone roaring. The episode's energy peaked when Casey Rocket hit the stage with his unique brand of humor, complete with wild eyes and tales of drug-induced flashbacks at the mall. His physical comedy had the audience in stitches, proving why he's quickly becoming a Kill Tony favorite.

But let's not forget the regulars who never fail to bring the house down. Cam Patterson and his observations about Utah's Civil War allegiance and mountain sightings were both insightful and hilarious. Meanwhile, Hans Kim made a triumphant return after a hiatus, tackling topics from Asian women in San Francisco to the pitfalls of sending nudes. And who could overlook William Montgomery, who closed the show with his signature blend of absurdity and sharp commentary, even managing to weave in a tale about a racist dead squirrel in his chimney. Every joke hit, every performance sizzled, and the audience was left wanting more. This episode truly showcased why Kill Tony remains the number one live podcast in the world.

In [21]:
display(Markdown(analysis_task.output_file))

The success of a comedian's set on Kill Tony is influenced by several factors, including delivery, punchlines, audience engagement, and panel interaction. Comedians like Casey Rocket, Jerry Carlin, Cam Patterson, Hans Kim, Kelly Quinn, and William Montgomery received positive reactions due to their strong delivery, clever writing, and ability to engage the audience. On the other hand, comedians like Leno Rodriguez, Andrew Ginsburg, and Matt Sturm experienced mixed or negative reactions, often due to weaker punchlines or a lack of connection with the audience.

In [22]:
display(Markdown(identify_talent_task.output_file))

**Promising Comedians with Standout Performances and Potential for Success**

1. **Casey Rocket:**
   - **Delivery:** Energetic and unpredictable, akin to a rollercoaster ride.
   - **Punchlines:** Layered and often escalate into absurdity.
   - **Audience Engagement:** Top-notch, plays off audience reactions.
   - **Panel Interaction:** Charms the panel with infectious enthusiasm and sharp wit.

2. **Jerry Carlin:**
   - **Delivery:** Laid-back and deadpan style.
   - **Punchlines:** Sharp, concise, and meticulously crafted.
   - **Audience Engagement:** Excellent timing, knows how to work the room.
   - **Panel Interaction:** Polished performance, appreciated for staying cool under pressure.

3. **Cam Patterson:**
   - **Delivery:** Confident and engaging, natural knack for storytelling.
   - **Punchlines:** Cleverly woven into longer narratives.
   - **Audience Engagement:** Shines with audience interaction, making each show feel personal.
   - **Panel Interaction:** Praised for originality and stage command.

4. **Hans Kim:**
   - **Delivery:** Smooth and understated.
   - **Punchlines:** Sharp social commentary delivered with a wry smile.
   - **Audience Engagement:** Reads the room well, adjusts pace and tone.
   - **Panel Interaction:** Intelligent humor, handles delicate topics with subtlety.

5. **Kelly Quinn:**
   - **Delivery:** Confident with impeccable comedic timing.
   - **Punchlines:** Everyday observations turned into laugh-out-loud moments.
   - **Audience Engagement:** Excellent rapport, dynamic and inclusive atmosphere.
   - **Panel Interaction:** Bold and seamlessly integrates crowd reactions.

6. **William Montgomery:**
   - **Delivery:** Frenetic and manic energy.
   - **Punchlines:** Rapid-fire, often absurd and unexpected.
   - **Audience Engagement:** Thrives on chaos, uses hecklers and reactions to fuel his act.
   - **Panel Interaction:** Refreshing and delightfully offbeat brand of comedy.

These comedians have demonstrated strong delivery, clever writing, and exceptional audience engagement, making them standout performers with potential for success in the comedy world.

In [26]:
from IPython.display import JSON

display(JSON(find_venue_task.output_file))

<IPython.core.display.JSON object>